In [1]:
import pyspark
import findspark
findspark.init('/opt/anaconda3/lib/python3.11/site-packages/pyspark')
from pyspark import SparkContext
from pyspark.sql.types import *
# pyspark.__version__

conf=pyspark.SparkConf().setMaster('local').setAppName("first")
sc = SparkContext(conf = conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 05:58:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 05:58:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc.stop()

In [3]:
#Create a session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Example").getOrCreate()

23/09/22 05:58:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
data = [(1,'a', 30), (2, "b", 66)]
userSchema = StructType([StructField("id", IntegerType()),
                        StructField("name", StringType()),
                        StructField("age", IntegerType())])

df = spark.createDataFrame(data, userSchema)

In [5]:
df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|   a| 30|
|  2|   b| 66|
+---+----+---+



In [6]:
data2 = [
    (1, "Alice", ["Reading", "Hiking"]),
    (2, "Bob", ["Swimming", "Gardening", "Painting"]),
    (3, "Charlie", ["Cooking"]),
    (4, "David", ["Photography", "Skiing", "Cooking"])
]

In [7]:
schema2 = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("hobbies", ArrayType(StringType(), True), True)
])


df2 = spark.createDataFrame(data2, schema2)

In [8]:
df2.show()

+---+-------+--------------------+
| id|   name|             hobbies|
+---+-------+--------------------+
|  1|  Alice|   [Reading, Hiking]|
|  2|    Bob|[Swimming, Garden...|
|  3|Charlie|           [Cooking]|
|  4|  David|[Photography, Ski...|
+---+-------+--------------------+



In [9]:
from pyspark.sql.functions import *
df3 = df2.select("id", "name", explode("hobbies").alias("hobby"))

In [10]:
df3.show()

+---+-------+-----------+
| id|   name|      hobby|
+---+-------+-----------+
|  1|  Alice|    Reading|
|  1|  Alice|     Hiking|
|  2|    Bob|   Swimming|
|  2|    Bob|  Gardening|
|  2|    Bob|   Painting|
|  3|Charlie|    Cooking|
|  4|  David|Photography|
|  4|  David|     Skiing|
|  4|  David|    Cooking|
+---+-------+-----------+



In [11]:
df3 = df2.withColumn("newHobby", explode("hobbies"))
df3.show()

+---+-------+--------------------+-----------+
| id|   name|             hobbies|   newHobby|
+---+-------+--------------------+-----------+
|  1|  Alice|   [Reading, Hiking]|    Reading|
|  1|  Alice|   [Reading, Hiking]|     Hiking|
|  2|    Bob|[Swimming, Garden...|   Swimming|
|  2|    Bob|[Swimming, Garden...|  Gardening|
|  2|    Bob|[Swimming, Garden...|   Painting|
|  3|Charlie|           [Cooking]|    Cooking|
|  4|  David|[Photography, Ski...|Photography|
|  4|  David|[Photography, Ski...|     Skiing|
|  4|  David|[Photography, Ski...|    Cooking|
+---+-------+--------------------+-----------+



In [12]:
df4 = df3.withColumn("Ingestion Time", current_timestamp())

In [13]:
df3.show(truncate = False)

+---+-------+-------------------------------+-----------+
|id |name   |hobbies                        |newHobby   |
+---+-------+-------------------------------+-----------+
|1  |Alice  |[Reading, Hiking]              |Reading    |
|1  |Alice  |[Reading, Hiking]              |Hiking     |
|2  |Bob    |[Swimming, Gardening, Painting]|Swimming   |
|2  |Bob    |[Swimming, Gardening, Painting]|Gardening  |
|2  |Bob    |[Swimming, Gardening, Painting]|Painting   |
|3  |Charlie|[Cooking]                      |Cooking    |
|4  |David  |[Photography, Skiing, Cooking] |Photography|
|4  |David  |[Photography, Skiing, Cooking] |Skiing     |
|4  |David  |[Photography, Skiing, Cooking] |Cooking    |
+---+-------+-------------------------------+-----------+



### Constructors Case Study

In [15]:
path = "/home/labuser/Desktop/Python/constructors.json"
df = spark.read.json(path)
df.show()

+-------------+--------------+-----------+-----------+--------------------+
|constructorId|constructorRef|       name|nationality|                 url|
+-------------+--------------+-----------+-----------+--------------------+
|            1|       mclaren|    McLaren|    British|http://en.wikiped...|
|            2|    bmw_sauber| BMW Sauber|     German|http://en.wikiped...|
|            3|      williams|   Williams|    British|http://en.wikiped...|
|            4|       renault|    Renault|     French|http://en.wikiped...|
|            5|    toro_rosso| Toro Rosso|    Italian|http://en.wikiped...|
|            6|       ferrari|    Ferrari|    Italian|http://en.wikiped...|
|            7|        toyota|     Toyota|   Japanese|http://en.wikiped...|
|            8|   super_aguri|Super Aguri|   Japanese|http://en.wikiped...|
|            9|      red_bull|   Red Bull|   Austrian|http://en.wikiped...|
|           10|   force_india|Force India|     Indian|http://en.wikiped...|
|           

In [17]:
df2 = df.withColumn("Ingestion Time", current_timestamp()).withColumn("path", input_file_name()).show()

+-------------+--------------+-----------+-----------+--------------------+--------------------+--------------------+
|constructorId|constructorRef|       name|nationality|                 url|      Ingestion Time|                path|
+-------------+--------------+-----------+-----------+--------------------+--------------------+--------------------+
|            1|       mclaren|    McLaren|    British|http://en.wikiped...|2023-09-22 05:59:...|file:///home/labu...|
|            2|    bmw_sauber| BMW Sauber|     German|http://en.wikiped...|2023-09-22 05:59:...|file:///home/labu...|
|            3|      williams|   Williams|    British|http://en.wikiped...|2023-09-22 05:59:...|file:///home/labu...|
|            4|       renault|    Renault|     French|http://en.wikiped...|2023-09-22 05:59:...|file:///home/labu...|
|            5|    toro_rosso| Toro Rosso|    Italian|http://en.wikiped...|2023-09-22 05:59:...|file:///home/labu...|
|            6|       ferrari|    Ferrari|    Italian|ht

In [18]:
df_final = df.withColumn("Ingestion Time", current_timestamp()).withColumn("path", input_file_name()).drop("url")

In [19]:
df_final.show()

+-------------+--------------+-----------+-----------+--------------------+--------------------+
|constructorId|constructorRef|       name|nationality|      Ingestion Time|                path|
+-------------+--------------+-----------+-----------+--------------------+--------------------+
|            1|       mclaren|    McLaren|    British|2023-09-22 05:59:...|file:///home/labu...|
|            2|    bmw_sauber| BMW Sauber|     German|2023-09-22 05:59:...|file:///home/labu...|
|            3|      williams|   Williams|    British|2023-09-22 05:59:...|file:///home/labu...|
|            4|       renault|    Renault|     French|2023-09-22 05:59:...|file:///home/labu...|
|            5|    toro_rosso| Toro Rosso|    Italian|2023-09-22 05:59:...|file:///home/labu...|
|            6|       ferrari|    Ferrari|    Italian|2023-09-22 05:59:...|file:///home/labu...|
|            7|        toyota|     Toyota|   Japanese|2023-09-22 05:59:...|file:///home/labu...|
|            8|   super_aguri|

In [21]:
df_final.write.mode("overwrite").parquet("/home/labuser/Desktop/Python/Processed Data/Constructor/constructors_parquet")

In [22]:
df_final.write.saveAsTable("Constructors")

In [24]:
df_final.write.option("path", "/home/labuser/Desktop/Python/Processed Data/Constructor Table").saveAsTable("constructor_table")